# A first look at a neural network
Chapter 2, Section 1 of Deep Learning with Python.

10-08-2021

https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/first_edition/2.1-a-first-look-at-a-neural-network.ipynb

## 1. 数据读取

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [49]:
#### 得用tensorflow.keras.datasets而不是keras.datasets
# 如果是import keras，那么可能还是keras.datasets
from tensorflow.keras.datasets import mnist
#### 构建train & test数据集
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images and train_labels form the "training set", the data that the model will learn from. The model will then be tested on the "test set", test_images and test_labels. Our images are encoded as Numpy arrays, and the labels are simply an array of digits, ranging from 0 to 9. There is a one-to-one correspondence between the images and the labels.

Let's have a look at the training data:

In [50]:
train_images.shape

(60000, 28, 28)

In [51]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [52]:
test_images.shape

(10000, 28, 28)

In [53]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

Our workflow will be as follow: first we will present our neural network with the training data, train_images and train_labels. The network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for test_images, and we will verify if these predictions match the labels from test_labels.

Let's build our network -- again, remember that you aren't supposed to understand everything about this example just yet.

In [54]:
from tensorflow.keras import models
from tensorflow.keras import layers

network = models.Sequential() # 创建empty的网络
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))#添加由relu激活的layers
network.add(layers.Dense(10, activation='softmax')) # 由softmax对应多个classification


The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some data comes in, and comes out in a more useful form. Precisely, layers extract representations out of the data fed into them -- hopefully representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers which will implement a form of progressive "data distillation"(渐进式的数据蒸馏). A deep learning model is like a sieve (筛子) for data processing, made of a succession of increasingly refined data filters -- the "layers".

Here our network consists of a sequence of two Dense layers, which are densely-connected (also called "fully-connected") neural layers. The second (and last) layer is a 10-way "softmax" layer, which means it will return an array of 10 probability scores(返回十个概率值) (summing to 1). Each score will be the probability that the current digit image belongs to one of our 10 digit classes.

To make our network ready for training, we need to pick three more things, as part of "compilation" step:(还需三个编译步骤)

1.A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be able to steer itself in the right direction.(朝着正确方向前进)

2.An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.(基于训练数据和损失函数来更新网络的机制)

3.Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly classified).

The exact purpose of the loss function and the optimizer will be made clear throughout the next two chapters.

In [55]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

## 数据预处理
Before training, we will preprocess our data by reshaping it into the shape that the network expects, and scaling it so that all values are in the [0, 1] interval. Previously, our training images for instance were stored in an array of shape (60000, 28, 28) of type uint8 with values in the [0, 255] interval. We transform it into a float32 array of shape (60000, 28 * 28) with values between 0 and 1.

In [56]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

We also need to categorically encode the labels(对数据标签编码), a step which we explain in chapter 3:

In [57]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

We are now ready to train our network, which in Keras is done via a call to the fit method of the network: we "fit" the model to its training data.

In [63]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)
# batch_size 会影响运行速度，
# batch_size=12 | batch_size=128
# Epoch 1/5 | 
# 5000/5000 | 469/469

Epoch 1/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0136 - accuracy: 0.9965
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0079 - accuracy: 0.9979
Epoch 3/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0049 - accuracy: 0.9987
Epoch 4/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0036 - accuracy: 0.9989
Epoch 5/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0027 - accuracy: 0.9994


Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over the training data.

We quickly reach an accuracy of 0.989 (i.e. 98.9%) on the training data. Now let's check that our model performs well on the test set too:

In [64]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 750us/step - loss: 0.1140 - accuracy: 0.9811


In [65]:
print('test_loss:', test_loss)
print('test_acc:', test_acc)

test_loss: 0.11398300528526306
test_acc: 0.9811000227928162


Our test set accuracy turns out to be 98.1% -- that's quite a bit lower than the training set accuracy. This gap between training accuracy and test accuracy is an example of "overfitting", the fact that machine learning models tend to perform worse on new data than on their training data. Overfitting will be a central topic in chapter 3.

This concludes our very first example -- you just saw how we could build and a train a neural network to classify handwritten digits, in less than 20 lines of Python code. In the next chapter, we will go in detail over every moving piece we just previewed, and clarify what is really going on behind the scenes. You will learn about "tensors", the data-storing objects going into the network, about tensor operations, which layers are made of, and about gradient descent, which allows our network to learn from its training examples.

In [ ]:
#Using the model to make predictions
test_digits = test_images[0:10]
predictions = network.predict(test_digits)
predictions[0]
predictions[0].argmax()
predictions[0][7]
test_labels[0]